In [5]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
import random
import pickle

iris = load_iris()
df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                  columns= iris['feature_names'] + ['target'])

X = df.iloc[:100,[0,2]].values
y = df.iloc[0:100,4].values
y = np.where(y == 0, -1, 1)

class Perceptron():
    
    def __init__(self, eta=0.01, n_iter=10):
        self.eta = eta
        self.n_iter = n_iter
    
    def fit(self, X, y):
        
        #self.w_ = np.zeros(1+X.shape[1])
        self.w_ = [random.uniform(-1.0, 1.0) for _ in range(1+X.shape[1])] 
        self.errors_ = []
        
        for _ in range(self.n_iter):
            errors = 0
            for xi, target in zip(X,y):
                #print(xi, target)
                update = self.eta*(target-self.predict(xi))
                #print(update)
                self.w_[1:] += update*xi
                self.w_[0] += update
                #print(self.w_)
                errors += int(update != 0.0)
            self.errors_.append(errors)
        return self
    
    def net_input(self, X):
        return np.dot(X, self.w_[1:])+self.w_[0]
    
    def predict(self, X):
        return np.where(self.net_input(X)>=0.0, 1, -1)

ppn = Perceptron()
ppn.fit(X,y)
with open('model.pkl', "wb") as picklefile:
    pickle.dump(ppn, picklefile)

In [11]:
%%file app.py

import pickle
import numpy as np
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/predict_get', methods=['GET'])
def get_prediction():
    sepal_length = float(request.args.get('sl'))
    petal_length = float(request.args.get('pl'))    
    features = [sepal_length, petal_length]
    with open('model.pkl',"rb") as picklefile:
        model = pickle.load(picklefile)
    predicted_class = int(model.predict(features))    
    return jsonify(features=features, predicted_class=predicted_class)

@app.route('/predict_post', methods=['POST'])
def post_predict():
    data = request.get_json(force=True)
    sepal_length = float(data.get('sl'))
    petal_length = float(data.get('pl'))
    
    features = [sepal_length, petal_length]
    with open('model.pkl',"rb") as picklefile:
        model = pickle.load(picklefile)
    predicted_class = int(model.predict(features))
    output = dict(features=features, predicted_class=predicted_class)
    return jsonify(output)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

Overwriting app.py


In [2]:
%%file Dockerfile
FROM python:3.11-slim-buster

WORKDIR /app

COPY requirements.txt requirements.txt

RUN pip install -r requirements.txt

COPY app.py .

ENV FLASK_APP=app

EXPOSE 8000
CMD ["flask", "run", "--host", "0.0.0.0", "--port", "8000"]

Writing Dockerfile


In [10]:
import requests
response = requests.get("http://127.0.0.1:5000/predict_get?sl=10.4&pl=2.6")
print(response.content)
json = {"sl":2.4, "pl":2.6}
response = requests.post("http://127.0.0.1:5000/predict_post", json=json)
print(response.content)

b'{"features":[10.4,2.6],"predicted_class":-1}\n'
b'{"features":[2.4,2.6],"predicted_class":1}\n'
